# What databases need to be downloaded:
* smartsheets - API is running no need
* RTR - aDataManagerTicketExport 
* RTR URL: https://rtr.tdr.tetratech.com/rs

# # What we are checking over view:

#### Multiple checks
* Soil data with share ppoint and Smart sheet comparasions(learning curve to read the out put)
* Soil- Location in smart sheets
* Soil -Sample date after recived from lab dates
* Soil -Sample approval but debris status still in soils
#### other checks
* Soil sample prio to debris finish
* ROE checks - ROE date is blank | ROE Verified is blank | Property Type is blank
* FSO checks - FSO sent to county is not blank with ROE type full program and FSO debris complete is blank
* ASB sample sent prior to ASB assessment
* ASB results prior to ASB sample sent date
* Debris start date is after debris finish date
* Missing lat and long
* Tree check - Checks for number of days from tree felling complete till ready for TISW
* Salmon - obsolete
* 811 A&M checks
* Bird nests
* Bridge CRossing

In [1]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import timedelta
import smartsheet

Variable

In [2]:
fileType_ex = ".xlsx"
fileType_csv = ".csv"
division = 'Northern Division' # will be opposite of the division we are checking. Help check for evrything NOT in Northern Division
division_rtr = 'NORTHERN DIVISION'
# grabs smartsheet data 
# central = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*" 
# south = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"

# Load in smartsheet using the API

In [3]:
# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Smart sheet APIs
# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API
Central.errors_as_exceptions(True)# Make sure we don't miss any errors

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)# Make sure we don't miss any errors

# load in Smartsheets to pandas data frame - A FUNCTION IS NEEDED
# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    """Runs through the smartsheets collection of values and create a Pandas data frame"""
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

In [4]:
# df_both.info(verbose=True, show_counts=True)

# Smart Sheets dowload upload from download folder and concate

### made better above

In [5]:
# #Central
# load = glob.glob(central+fileType_ex)
# df = pd.read_excel(max(load, key=os.path.getctime), parse_dates=True)

In [6]:
# # South Central
# load2 = glob.glob(south+fileType_ex)
# df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True)

#### Concate Central and South Central

In [7]:
# df_both = pd.concat([df,df2])

In [8]:
# df_both.info(verbose=True)

# Fucntions

In [9]:
def compare(df, comp1=None, comp2=None, comp3=None):
    """Compares columns in our dataframe"""
    if comp3 == None:
        if df[comp1] == df[comp2]:
            return 1
        else:
            return 0
    else:
        if df[comp1] == df[comp2] and df[comp2] == df[comp3]:
            return 1
        else:
            return 0

        
        
def greater(df, comp1=None, comp2=None):
    """Checks for dates greater than another"""
    if df[comp1] > df[comp2]:
        return "Greater than"
    else:
        return "Less Than"

# Soil Share Point Data

In [10]:
path_soilTracker = r"C:\Users\jacque.trahan\Downloads\Soil Sampling Tracker*"
load_soil = glob.glob(path_soilTracker + fileType_csv)
soils_sharePoint = pd.read_csv(max(load_soil, key=os.path.getctime), usecols=['APN', 'APN Status', 'Field Notes', 'Tt Recommendation', 
                                                                       'State Recommendation', 'State Rec Details',
                                                                       'Access Issue', 'DTSC Mercury Site','State Recommendation',
                                                                      'State Rec Details','State Rec Date','QC State Recs','ExpectedResultsDate',
                                                                      'Current Sample Event','Division', 'Recs Notes'])

# RTR data import

In [11]:
path = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load_rtr = glob.glob(path+fileType_ex)
df_RTR = pd.read_excel(max(load_rtr, key=os.path.getctime))

df_RTR['Zone Name'] = df_RTR['Zone Name'].str[:3] + "-" + df_RTR['Zone Name'].str[3:6] + "-" + df_RTR['Zone Name'].str[6:]
df_RTR['Zone Name'] = df_RTR['Zone Name'].str.replace('167-310-2500', '167-310-25-00')
df_RTR['Service Code'] = df_RTR['Service Code'].str.strip()


# Check ROE dates

In [12]:
ROE_dates = df_both[['APN_ROW Segment','Street #', 'Street Name', 'County','Debris Crew', 'Structural Status',
                         'ROE Date', 'ROE Verified',
                    'Property Type', ]]
ROE_dates = ROE_dates[(ROE_dates['ROE Date'].isnull()) | 
                     (ROE_dates['ROE Verified'].isnull()) | 
                    (ROE_dates['Property Type'].isnull())]

ROE_dates.set_index('APN_ROW Segment', inplace=True)
ROE_dates.to_excel('ROE Random Checks.xlsx')

# FSO Check

In [13]:
FSO = df_both.copy()

FSO = FSO[(~FSO['FSO Sent to County'].isnull()) & (FSO['ROE Type'] == "Full Program ROE (debris + trees)") &
          (FSO['Debris FSO Complete'].isnull())]

FSO = FSO[['APN_ROW Segment','Street #', 'Street Name', 'County', 'Structural Status','ROE Type',
           'Erosion Control Complete', 'Erosion Control Waiver',
          'Debris FSO Complete', 'FSO Complete', 'FSO Sent to County']]

FSO.set_index('APN_ROW Segment', inplace=True)
FSO.to_excel('FSO Check.xlsx')

# ASB sample sent prior to ASB assessment

In [14]:
asb_sample = df_both.copy()
asb_sample = asb_sample[['APN_ROW Segment','Street #', 'Street Name', 'County',
                         'ASB Assessment', 'ASB Sample Sent']]
asb_sample = asb_sample[(asb_sample['ASB Assessment'] > asb_sample['ASB Sample Sent'])]

asb_sample.set_index('APN_ROW Segment', inplace=True)
asb_sample.to_excel('ASB sample sent prior to ASB assessment.xlsx')

# ASB results prior to ASB sample sent date


In [15]:
asb_res = df_both.copy()
asb_res = asb_res[['APN_ROW Segment','Street #', 'Street Name', 'County',
                   'ASB Sample Sent','ASB Results Received', 'ASB Results']]
asb_res = asb_res[(asb_res['ASB Sample Sent'].isna()) & (asb_res['ASB Results'].notna())]

asb_res.set_index('APN_ROW Segment', inplace=True)
asb_res.to_excel('ASB results prior to ASB sample sent date.xlsx')

# Debris start after to debris finish
### Check Debris dates

In [16]:
df_both['Debris Finish'].dtypes

dtype('O')

In [17]:
Debris_dates = df_both.copy()


In [18]:
Debris_dates = df_both.copy()
Debris_dates = Debris_dates[['APN_ROW Segment','Street #', 'Street Name', 'County',
                          'Debris Start', 'Debris Finish', 'State Rep Sign-Off for DR']]

Debris_dates['Debris Start'] = pd.to_datetime(Debris_dates['Debris Start'])
Debris_dates['Debris Finish'] = pd.to_datetime(Debris_dates['Debris Finish'])

Debris_dates.insert(Debris_dates.columns.get_loc('State Rep Sign-Off for DR')+1, "Finish start before Debris end",
                   Debris_dates.apply(greater,comp1='Debris Start', comp2='Debris Finish', axis=1))
Debris_dates.set_index('APN_ROW Segment', inplace=True)
Debris_dates.to_excel('Check debris start finish and state sign off dates.xlsx')

# Soil Checks

## Soil sample prior to debris finish

In [19]:
soil_debris = df_both.copy()
soil_debris = soil_debris[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
                           'Soil Sample', 'Debris Finish']]

soil_debris = soil_debris[(soil_debris['Soil Sample'] < soil_debris['Debris Finish'])]

soil_debris.set_index('APN_ROW Segment', inplace=True)
soil_debris.to_excel("Soil prior to debris Finish.xlsx")

## Multiple checks
* Location in smart sheets
* Sample date after recived from lab dates
* Sample approval but debris status still in soils

In [20]:
soilTak_soilRec = df_both.copy()
soilTak_soilRec = soilTak_soilRec[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','Debris Crew',
                                   'Soil Sample', 'Soil Sample Results Rec\'d from Lab',"Soil Sample Results",
                                  'Soil Boring','Soil Boring Results Rec\'d from Lab', 'Soil Boring Results',
                                  'Soil Re-scrape',"Re-scrape Results Rec'd from Lab", "Re-scrape Results",
                                  'Soil Re-scrape 2',"Re-scrape 2 Results Rec'd from Lab", "Re-scrape 2 Results",
                                  'Soil Rescrape 3',"Re-scrape 3 Results Rec'd from Lab", 'Re-scrape 3 Results',
                                  "Soil Samples Approved"]]



#### See where we are in Smartsheet with soils

In [21]:
def soil_location(df):
    """Prints out column header if the cell is not blank"""
    if pd.notna(df["Soil Samples Approved"]):
        return "Soil Samples Approved"
    
    elif pd.notnull(df['Re-scrape 3 Results']):
        return 'Re-scrape 3 Results'
    
    elif pd.notnull(df["Re-scrape 3 Results Rec'd from Lab"]):
        return "Re-scrape 3 Results Rec'd from Lab"
    
    elif pd.notnull(df["Re-scrape 2 Results"]):
        return "Re-scrape 2 Results"
    
    elif pd.notnull(df["Re-scrape 2 Results Rec'd from Lab"]):
        return "Re-scrape 2 Results Rec'd from Lab"
    
    elif pd.notnull(df['Soil Re-scrape 2']):
        return 'Soil Re-scrape 2'
    
    elif pd.notnull(df["Re-scrape Results"]):
        return "Re-scrape Results"
    
    elif pd.notnull(df["Re-scrape Results Rec'd from Lab"]):
        return "Re-scrape Results Rec'd from Lab"
    
    elif pd.notnull(df['Soil Re-scrape']):
        return 'Soil Re-scrape'
    
    elif pd.notnull(df['Soil Boring Results']):
        return 'Soil Boring Results'
    
    elif pd.notnull(df['Soil Boring Results Rec\'d from Lab']):
        return 'Soil Boring Results Rec\'d from Lab'
    
    elif pd.notnull(df['Soil Boring']):
        return 'Soil Boring'
    
    elif pd.notnull(df["Soil Sample Results"]):
        return "Soil Sample Results"
    
    elif pd.notnull(df['Soil Sample Results Rec\'d from Lab']):
        return 'Soil Sample Results Rec\'d from Lab'
    
    elif pd.notnull(df['Soil Sample']):
        return 'Soil Sample'
    


In [22]:
soilTak_soilRec.insert(soilTak_soilRec.columns.get_loc("Soil Samples Approved")+1, "Location in smartsheets",
                      soilTak_soilRec.apply(soil_location, axis=1))

### Merge with the Soil Share point with the following columns
* APN
* Field Notes
* Tt Recommendation
* State Recommendation
* State Rec Details
* Access Issue
* DTSC Mercury Site

In [23]:
soils_sharePoint = soils_sharePoint[~(soils_sharePoint['Division'].isin([division]))]

In [24]:
Soil_final = soilTak_soilRec.merge(soils_sharePoint,
                              how='outer',
                              left_on='APN_ROW Segment',
                              right_on='APN')

### Check for date discrepancies

In [25]:
def Soil_Sample(df):
    if df['Soil Sample'] > df['Soil Sample Results Rec\'d from Lab']:
        return 'Check Soil Sample Dates'
    else:
        return 'Match'
    
def Soil_Boring(df):
    if df['Soil Boring'] > df['Soil Boring Results Rec\'d from Lab']:
        return 'Check Soil Boring Dates'
    else:
        return 'Match'
    
def Soil_Rescrape(df):
    if df['Soil Re-scrape'] > df["Re-scrape Results Rec'd from Lab"]:
        return 'Check Soil Re-scrape Dates'
    else:
        return 'Match'
def Soil_Rescrape2(df):    
    if df['Soil Re-scrape 2'] > df["Re-scrape 2 Results Rec'd from Lab"]:
        return 'Check Soil Re-scrape 2 Dates'
    else:
        return 'Match'

def Soil_Rescrape3(df):    
    if df['Soil Rescrape 3'] > df["Re-scrape 3 Results Rec'd from Lab"]:
        return 'Check Soil Rescrape 3 Dates'
    else:
        return 'Match'

In [26]:
Soil_final = Soil_final.copy()
Soil_final = Soil_final[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','Debris Crew',
                         "Location in smartsheets", 'APN Status',
                         'Soil Sample', 'Soil Sample Results Rec\'d from Lab',"Soil Sample Results",
                         'Recs Notes','Current Sample Event','ExpectedResultsDate',"Soil Samples Approved",
                         
                      'Soil Boring','Soil Boring Results Rec\'d from Lab', 'Soil Boring Results',
                         
                      'Soil Re-scrape',"Re-scrape Results Rec'd from Lab", "Re-scrape Results",
                                             
                      'Soil Re-scrape 2',"Re-scrape 2 Results Rec'd from Lab", "Re-scrape 2 Results",
                                                
                      'Soil Rescrape 3',"Re-scrape 3 Results Rec'd from Lab", 'Re-scrape 3 Results',
                       
                         
                        'Field Notes', 'Tt Recommendation', 'State Recommendation', 'State Rec Details',
                         'Access Issue', 'DTSC Mercury Site',
                         'State Rec Details','State Rec Date','QC State Recs',
                         'Division']]

In [27]:
Soil_final['Soil Sample'] = pd.to_datetime(Soil_final['Soil Re-scrape 2'])



Soil_final['Soil Re-scrape 2'] = Soil_final['Soil Re-scrape 2'].fillna(pd.NaT)
Soil_final["Re-scrape 2 Results Rec'd from Lab"] = Soil_final["Re-scrape 2 Results Rec'd from Lab"].fillna(pd.NaT)



In [28]:
Soil_final['Soil Sample'] = pd.to_datetime(Soil_final['Soil Sample'])
Soil_final['Soil Sample Results Rec\'d from Lab'] = pd.to_datetime(Soil_final['Soil Sample Results Rec\'d from Lab'])

Soil_final['Soil Boring'] = pd.to_datetime(Soil_final['Soil Boring'])
Soil_final['Soil Boring Results Rec\'d from Lab'] = pd.to_datetime(Soil_final['Soil Boring Results Rec\'d from Lab'])

Soil_final['Soil Re-scrape'] = pd.to_datetime(Soil_final['Soil Re-scrape'])
Soil_final["Re-scrape Results Rec'd from Lab"] = pd.to_datetime(Soil_final["Re-scrape Results Rec'd from Lab"])

Soil_final['Soil Re-scrape 2'] = pd.to_datetime(Soil_final['Soil Re-scrape 2'])
Soil_final["Re-scrape 2 Results Rec'd from Lab"] = pd.to_datetime(Soil_final["Re-scrape 2 Results Rec'd from Lab"])

Soil_final['Soil Rescrape 3'] = pd.to_datetime(Soil_final['Soil Rescrape 3'])
Soil_final["Re-scrape 3 Results Rec'd from Lab"] = pd.to_datetime(Soil_final["Re-scrape 3 Results Rec'd from Lab"])

In [29]:
# sample date check
Soil_final.insert(Soil_final.columns.get_loc("Soil Samples Approved")+1, 'Soil_Sample Date checks-M',
                 Soil_final.apply(Soil_Sample, axis=1))

# Soil_Boring Check
Soil_final.insert(Soil_final.columns.get_loc('Soil Boring Results')+1, 'Soil_Boring Date checks-M',
                 Soil_final.apply(Soil_Boring, axis=1))

#Soil_Rescrape
Soil_final.insert(Soil_final.columns.get_loc("Re-scrape Results")+1, 'Soil_Rescrape Date checks-M',
                 Soil_final.apply(Soil_Rescrape, axis=1))

#Soil_Rescrape2
Soil_final.insert(Soil_final.columns.get_loc("Re-scrape 2 Results")+1, 'Soil_Rescrape2 Date checks-M',
                 Soil_final.apply(Soil_Rescrape2, axis=1))

#Soil_Rescrape3
Soil_final.insert(Soil_final.columns.get_loc('Re-scrape 3 Results')+1, 'Soil_Rescrape3 Date checks-M',
                 Soil_final.apply(Soil_Rescrape3, axis=1))

Soil_final.set_index('APN_ROW Segment', inplace=True)
Soil_final.to_excel('Check Soil APN status vs structual status and current location in smartsheets.xlsx')

# long and lats missing

In [30]:
lon_lat = df_both.copy()
lon_lat = lon_lat[['APN_ROW Segment','Street #', 'Street Name', 'County','Latitude', 'Longitude']]

lon_lat = lon_lat[lon_lat['Latitude'].isna() | lon_lat['Longitude'].isna()]

lon_lat.set_index('APN_ROW Segment', inplace=True)
lon_lat.to_excel("Missing long and lat.xlsx")

# TREE CHECKS

### import google sheets

In [31]:
#Sheet ID comes right after /d/ in the URL
Sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# name of tab you want to use
sheets_name = 'Tree RTR Tracker'

# this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
#gid # in URL lets you pick the sheet you want. Will need to look for that. 
url = f'https://docs.google.com/spreadsheets/d/{Sheet_ID}/export?format=csv#id={sheets_name}'

google_Qc = pd.read_csv(url, header=1)
google_Qc = google_Qc[['APN_ROW Segment', 'ECC Tree Removal Confirmation']]

In [32]:
# checks if tree felling completed has a date and tree removal competed does not
TREE = df_both.copy()
TREE = TREE[(~TREE['Tree Felling Complete'].isnull()) & (TREE['Tree Removal Complete (ISW)'].isnull())]

#number of days from tree removal completed
TREE['# of days pasted Tree Felling Complete Date'] = pd.Timestamp.now().normalize()  - pd.to_datetime(TREE['Tree Felling Complete'], errors='coerce')


#Merge smartsheets and google QC
Tree_final = TREE.merge(google_Qc,
                       left_on='APN_ROW Segment',
                       right_on='APN_ROW Segment',
                       how='left')

# number of days Since notificed APN was ready for TISW
Tree_final['# of days pasted Ready for ECC/Anvil TISW update'] = pd.Timestamp.now().normalize()  - pd.to_datetime(Tree_final['ECC Tree Removal Confirmation'], errors='coerce')

Tree_final.set_index('APN_ROW Segment', inplace=True)
Tree_final[['Street #', 'Street Name', 'Structural Status','County', 'Tree Crew',
            'Tree Felling Complete', 'Tree Removal Complete (ISW)','ECC Tree Removal Confirmation',
            '# of days pasted Tree Felling Complete Date','# of days pasted Ready for ECC/Anvil TISW update',
            'Number of Trees',
           ]].to_excel("Number of days TISW has been Pending.xlsx")

# Check Tree removal and TISW dates
https://docs.google.com/spreadsheets/d/1wF0GLNJG0dKTnkT0fEuC-kAIU4pzrr_IyhLnJ5iEZ7s/edit#gid=1185136840

In [33]:
Planning_sheet_id = '1wF0GLNJG0dKTnkT0fEuC-kAIU4pzrr_IyhLnJ5iEZ7s'
Planning_sheet_name = '1185136840'

#gid # in URL lets you pick the sheet you want. Will need to look for that.
url_planning = f"https://docs.google.com/spreadsheets/d/{Planning_sheet_id}/export?gid={Planning_sheet_name}&format=csv"
planning_google = pd.read_csv(url_planning)


In [34]:
# final_google = planning_google.merge(df_both,
#                                     left_on='APN_ROW Segment',
#                                     right_on='APN_ROW Segment',
#                                     how='left',
#                                     suffixes=("_GG","_SS"))

# final_google = final_google[['APN_ROW Segment', 'Tree Removal Complete_GG','Tree Removal Complete_SS']]
# final_google['Tree Removal Complete_GG'] = pd.to_datetime(final_google['Tree Removal Complete_GG']).dt.date
# final_google['Tree Removal Complete_SS'] = pd.to_datetime(final_google['Tree Removal Complete_SS']).dt.date

# final_google['Match Check'] = (final_google['Tree Removal Complete_GG'] == final_google['Tree Removal Complete_SS'])

# final_google = final_google[final_google['Match Check'] == False]
# final_google.to_excel('Check TREE removal complete date.xlsx')

# Check for Salmon

In [35]:
# salmon = df_both.copy()
# salmon = salmon[salmon['Salmon Habitat'].notna()][['APN_ROW Segment','Street #', 'Street Name', 'Structural Status','County', 'Tree Crew','Salmon Habitat']]
# salmon['Salmon Habitat'] = np.where(salmon['Salmon Habitat'] == 1,'Has Salmon', "")
# salmon.set_index('APN_ROW Segment', inplace=True)
# salmon.to_excel('Check Salmon Habit.xlsx')

 # Temp signage checks - RTR vs smartsheets

In [36]:
# #RTR data set up
# sign = df_RTR.copy()
# sign = sign[['Zone Name','Is Void', 'GIS Zone1', 'Service Code', 'Unit Count']]
# sign = sign[(sign['Service Code'].isin(["67A"])) & ~(sign['GIS Zone1'].isin([division_rtr])) & (sign['Is Void']== False)]


In [37]:
# # Smartsheets set up
# sign_SS = df_both.copy()
# sign_SS = sign_SS[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status', 'Debris Crew', 'Tree Crew',
#                         '811 Called (A&M)']]
# final_sign = sign_SS.merge(sign,
#                           left_on='APN_ROW Segment',
#                           right_on='Zone Name',
#                           how='outer')
# final_sign.to_excel('811 A&M Ticket checks RTR vs Smart Sheets.xlsx')

# Check for Abatment property statuses:

In [38]:
abatment = df_both.copy()
abatment = abatment[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','ROE Type', 'Debris Crew',
                     'Tree Crew',
            'Debris Start', 'Debris Finish', 'Soil Samples Approved', 'FSO Complete',
                     'Haz Trees Present?', 'Tree Felling Start','Tree Removal Complete (ISW)', 'Tree FSW',
                     'ROE Verified', 'ROE Date']]
abatment['Abatment Expires'] = pd.to_datetime(abatment['ROE Date']).dt.date + timedelta(days=30)

abatment.set_index('APN_ROW Segment',inplace=True)
abatment = abatment[abatment['ROE Verified'] == 'Abatement'].to_excel('Abatment Checks.xlsx')

In [39]:
# df_both.info(verbose=True)

# check holds and their status

In [40]:
Holds = df_both.copy()
Holds = Holds[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','ROE Type',
               'Debris Crew', 'Tree Crew',
            'Debris Start', 'Debris Finish',
               'Soil Samples Approved', 'FSO Complete', 'FSO Sent to County',
                     'Tree FSW','Hold', 'Hold Reason',
                     'ROE Verified']]

Holds = Holds[Holds['Hold'] == True].to_excel("Holds checks.xlsx")

# Check Bird Nests that are active

In [41]:
birds = df_both.copy()
birds = birds[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
              'Nesting Bird Survey Date', 'Nesting Bird Survey Results', 'Hold']]
birds = birds[(birds['Nesting Bird Survey Results'] == "Active Nest") & (birds['Hold'] != 1.0)]

birds.set_index('APN_ROW Segment', inplace=True)
birds.to_excel('Active Birds nest not on hold.xlsx')

# bridge/crossing checks:

In [42]:
bridge = df_both.copy()
bridge = bridge[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
                'OES EHP Attention Required',
                 'Water crossing/bridge needed?', 'Water crossing/bridge install', 'Water crossing/bridge removal']]
bridge = bridge[bridge['OES EHP Attention Required'].notna()]
bridge.set_index('APN_ROW Segment', inplace=True)
bridge.to_excel('Check for bridge updates.xlsx')